In [1]:
!pwd
import sys

C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\notebook\MLPClassifier.fit\20201111


In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
sys.executable

'c:\\users\\81908\\appdata\\local\\pypoetry\\cache\\virtualenvs\\tfgpu-ehdmne1y-py3.8\\scripts\\python.exe'

In [3]:
#!pip install adabelief-tf==0.1.0

In [4]:
import sys

# sys.path.append("../input/iterative-stratification/iterative-stratification-master")

In [5]:
## GPU使わない
#import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [6]:
import os
import glob
import warnings

import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow_addons as tfa

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [7]:
from funcs import *

In [8]:
logger = Logger("./")

In [9]:
import tensorflow as tf
from adabelief_tf import AdaBeliefOptimizer


def get_optimizers(
    choice_optim="sgd",
    lr=0.01,
    decay=0.0,
    momentum=0.9,
    nesterov=True,  # SGD
    rmsprop_rho=0.9,  # RMSprop
    adadelta_rho=0.95,  # Adadelta
    beta_1=0.9,
    beta_2=0.999,
    amsgrad=False,  # Adam, Adamax, Nadam
    total_steps=0,
    warmup_proportion=0.1,
    min_lr=0.0,  # RAdam
    *args,
    **kwargs,
):
    """
    オプティマイザを取得する
    引数のchoice_optim は 'sgd', 'rmsprop', 'adagrad', 'adadelta', 'adam', 'adamax', 'nadam', 'adabelief', 'radam' のいずれかを指定する
    """
    optim = ""
    if choice_optim == "sgd":
        optim = tf.optimizers.SGD(
            lr=lr, momentum=momentum, decay=decay, nesterov=nesterov
        )
    elif choice_optim == "rmsprop":
        optim = tf.optimizers.RMSprop(lr=lr, rho=rmsprop_rho, decay=decay)
    elif choice_optim == "adagrad":
        optim = tf.optimizers.Adagrad(lr=lr, decay=decay)
    elif choice_optim == "adadelta":
        optim = tf.optimizers.Adadelta(lr=lr, decay=decay, rho=adadelta_rho)
    elif choice_optim == "adam":
        optim = tf.optimizers.Adam(
            lr=lr, decay=decay, beta_1=beta_1, beta_2=beta_2, amsgrad=amsgrad
        )
    elif choice_optim == "adamax":
        optim = tf.optimizers.Adamax(lr=lr, decay=decay, beta_1=beta_1, beta_2=beta_2)
    elif choice_optim == "nadam":
        if decay == 0.0:
            # Nadam はschedule_decay=0.004 がデフォルト値
            schedule_decay = 0.004
        else:
            schedule_decay = decay
        optim = tf.optimizers.Nadam(
            lr=lr, schedule_decay=schedule_decay, beta_1=beta_1, beta_2=beta_2
        )
    elif choice_optim == "radam":
        if lr == 0.0:
            lr = 0.001
        optim = tfa.optimizers.RectifiedAdam(
            learning_rate=lr,
            weight_decay=decay,
            beta_1=beta_1,
            beta_2=beta_2,
            amsgrad=amsgrad,
            total_steps=total_steps,
            warmup_proportion=warmup_proportion,
            min_lr=min_lr,
        )
    elif choice_optim == "adabelief":
        optim = AdaBeliefOptimizer(learning_rate=lr,)
    return optim

In [10]:
def build_model(
    input_dim,
    output_dim,
    activation="relu",
    bias_initializer="zeros",
    kernel_initializer="glorot_uniform",
    label_smoothing=0.0,
    lr=1e-03,
    n_layers=3,
    n_units="auto",
    pretrained_model_path=None,
    rate=0.0,
    skip=False,
    stddev=0.0,
    other_params={
        "optimizer": "adabelief",
        "is_lookahead": False,
        "is_swa": False,
        "is_weightnorm": True,
        "nn_order": "batch-act-drop",
        "drop_act_rate": 0.0,
    },
):
    if n_units == "auto":
        n_units = 0.5 * (input_dim + output_dim)

    inputs = tf.keras.layers.Input(shape=input_dim, name="I")

    x = inputs
    if stddev > 0.0:
        x = tf.keras.layers.GaussianNoise(stddev, name="GN")(x, training=True)

    x = tf.keras.layers.Dropout(rate, name="D0")(x)

    for i in range(n_layers - 2):
        if other_params["is_weightnorm"]:
            x = tfa.layers.WeightNormalization(
                tf.keras.layers.Dense(n_units, kernel_initializer=kernel_initializer),
                name=f"WN{i + 1}",
            )(x)
        else:
            x = tf.keras.layers.Dense(
                n_units, kernel_initializer=kernel_initializer, name=f"DN{i + 1}"
            )(x)

        # batch-act-drop
        if other_params["nn_order"] == "batch-act-drop":
            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

        # batch-drop-act
        if other_params["nn_order"] == "batch-drop-act":
            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

        # drop-act-batch
        if other_params["nn_order"] == "drop-act-batch":
            if other_params["drop_act_rate"] <= 0.0:
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)
            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

        # drop-batch-act
        if other_params["nn_order"] == "drop-batch-act":
            if other_params["drop_act_rate"] <= 0.0:
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)
            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

        # act-batch-drop
        if other_params["nn_order"] == "act-batch-drop":
            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)
            if other_params["drop_act_rate"] <= 0.0:
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

        # act-drop-batch
        if other_params["nn_order"] == "act-drop-batch":
            if other_params["drop_act_rate"] > 0.0:
                x = DropActivation(
                    activation, name=f"A{i + 1}", rate=other_params["drop_act_rate"]
                )(x)
            else:
                x = tf.keras.layers.Activation(activation, name=f"A{i + 1}")(x)

            if skip and i > 0 and i % 2 == 0:
                x = x + shortcut

            if other_params["drop_act_rate"] <= 0.0:
                x = tf.keras.layers.Dropout(rate, name=f"D{i + 1}")(x)
            x = tf.keras.layers.BatchNormalization(name=f"BN{i + 1}")(x)

            if skip and i % 2 == 0:
                shortcut = x

    if other_params["is_weightnorm"]:
        x = tfa.layers.WeightNormalization(
            tf.keras.layers.Dense(output_dim, bias_initializer=bias_initializer),
        )(x)
    else:
        x = tf.keras.layers.Dense(output_dim, bias_initializer=bias_initializer)(x)

    outputs = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)

    if pretrained_model_path is not None:
        model.load_weights(pretrained_model_path, by_name=True, skip_mismatch=True)

    loss = tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing)

    optimizer = get_optimizers(choice_optim=other_params["optimizer"], lr=lr)
    if other_params["is_lookahead"]:
        optimizer = tfa.optimizers.Lookahead(optimizer)
    if other_params["is_swa"]:
        optimizer = tfa.optimizers.SWA(optimizer)

    model.compile(loss=loss, optimizer=optimizer)

    return model

In [11]:
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.model_selection._split import _BaseKFold


class MultilabelGroupStratifiedKFold(_BaseKFold):
    def __init__(self, n_splits=5, random_state=None, shuffle=False):
        super().__init__(n_splits=n_splits, random_state=random_state, shuffle=shuffle)

    def _iter_test_indices(self, X=None, y=None, groups=None):
        cv = MultilabelStratifiedKFold(
            n_splits=self.n_splits,
            random_state=self.random_state,
            shuffle=self.shuffle,
        )

        value_counts = groups.value_counts()
        regular_index = value_counts.loc[
            (value_counts == 6) | (value_counts == 12) | (value_counts == 18)
        ].index.sort_values()
        irregular_index = value_counts.loc[
            (value_counts != 6) & (value_counts != 12) & (value_counts != 18)
        ].index.sort_values()

        group_to_fold = {}
        tmp = Y.groupby(groups).mean().loc[regular_index]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            group_to_fold.update({group: fold for group in tmp.index[test]})

        sample_to_fold = {}
        tmp = Y.loc[groups.isin(irregular_index)]

        for fold, (_, test) in enumerate(cv.split(tmp, tmp)):
            sample_to_fold.update({sample: fold for sample in tmp.index[test]})

        folds = groups.map(group_to_fold)
        is_na = folds.isna()
        folds[is_na] = folds[is_na].index.map(sample_to_fold).values

        for i in range(self.n_splits):
            yield np.where(folds == i)[0]

In [12]:
def preprocess(
    X,
    Y,
    Y_nonscored,
    X_test,
    preprocess_params={
        "is_clip": True,
        "is_stat": True,
        "is_add_nonscored": True,
        "is_c_squared": False,
        "is_c_abs": False,
        "is_g_valid": False,
        "pca_n_components_g": 0,
        "pca_n_components_c": 0,
        "is_scaling": False,
        "quantile_n_components": 0,
        "fe_cluster_n_clusters_g": 0,
        "fe_cluster_n_clusters_c": 0,
        "is_fit_train_only": True,
        "variance_threshold": 0.0,
    },
):
    if preprocess_params["is_clip"]:
        clipped_features = ClippedFeatures()
        X = clipped_features.fit_transform(X)

        # with open("clipped_features.pkl", "wb") as f:
        #    pickle.dump(clipped_features, f)

    if preprocess_params["quantile_n_components"] > 0:
        X, X_test = fe_quantile_transformer(
            X,
            X_test,
            n_quantiles=preprocess_params["quantile_n_components"],
            random_state=0,
        )

    if preprocess_params["fe_cluster_n_clusters_g"] > 0:
        X, X_test, _ = fe_cluster(
            X,
            X_test,
            n_clusters_g=preprocess_params["fe_cluster_n_clusters_g"],
            n_clusters_c=preprocess_params["fe_cluster_n_clusters_c"],
            random_state=123,
            is_fit_train_only=preprocess_params["is_fit_train_only"],
        )

    if preprocess_params["is_c_squared"]:
        X, X_test, _ = c_squared(X, X_test)

    if preprocess_params["is_c_abs"]:
        X, X_test, _ = c_abs(X, X_test)

    if preprocess_params["is_g_valid"]:
        X, X_test, _ = g_valid(X, X_test)

    if preprocess_params["pca_n_components_g"] > 0:
        X, X_test, _ = fe_pca(
            X,
            X_test,
            n_components_g=preprocess_params["pca_n_components_g"],
            n_components_c=preprocess_params["pca_n_components_c"],
            random_state=123,
            is_fit_train_only=preprocess_params["is_fit_train_only"],
        )

    if preprocess_params["is_scaling"]:
        X, X_test, _ = scaling(
            X, X_test, is_fit_train_only=preprocess_params["is_fit_train_only"],
        )

    if preprocess_params["is_stat"]:
        c_prefix = "c-"
        g_prefix = "g-"
        c_columns = X.columns.str.startswith(c_prefix)
        g_columns = X.columns.str.startswith(g_prefix)
        X_c = compute_row_statistics(X.loc[:, c_columns], prefix=c_prefix)
        X_g = compute_row_statistics(X.loc[:, g_columns], prefix=g_prefix)
        X = pd.concat([X, X_c, X_g], axis=1)

    if preprocess_params["variance_threshold"] > 0.0:
        X, X_test = fe_variance_threshold(
            X,
            X_test,
            get_features(X),
            threshold=preprocess_params["variance_threshold"],
            is_fit_train_only=preprocess_params["is_fit_train_only"],
        )

    if preprocess_params["is_add_nonscored"]:
        Y_nonscored = Y_nonscored.loc[:, Y_nonscored.sum(axis=0) > 0]

        Y = pd.concat([Y, Y_nonscored], axis=1)

    return X, Y, Y_nonscored

In [13]:
def pretrain(X, Y_nonscored, is_drug_kfold=True):
    bias_initializer = -Y_nonscored.mean(axis=0).apply(np.log).values
    bias_initializer = tf.keras.initializers.Constant(bias_initializer)

    # Y_pred = np.zeros((train_size, n_classes_nonscored))
    # Y_pred = pd.DataFrame(Y_pred, columns=Y_nonscored.columns, index=X.index)

    for i in range(n_seeds):
        set_seed(seed=i)

        if is_drug_kfold:
            cv = MultilabelGroupStratifiedKFold(
                n_splits=n_splits, random_state=i, shuffle=shuffle
            )
            cv_split = cv.split(X, Y_nonscored, groups)
        else:
            cv = MultilabelStratifiedKFold(
                n_splits=n_splits, random_state=i, shuffle=shuffle
            )
            cv_split = cv.split(X, Y_nonscored)

        for j, (train, valid) in tqdm(enumerate(cv_split)):

            #model_path = f"model_nonscored_seed_{i}_fold_{j}.h5"
            model_path = f"{out_dir}/model_nonscored_seed_{i}_fold_{j}.h5"

            K.clear_session()
            model = build_model(
                n_features,
                n_classes_nonscored,
                bias_initializer=bias_initializer,
                **params,
            )

            callbacks = build_callbacks(model_path, factor=factor, patience=patience)
            history = model.fit(
                X.iloc[train],
                Y_nonscored.iloc[train],
                callbacks=callbacks,
                validation_data=(X.iloc[valid], Y_nonscored.iloc[valid]),
                **fit_params,
            )

            # model.load_weights(model_path)

            # Y_pred.iloc[valid] += predict(model, X.iloc[valid], **predict_params) / n_seeds

    # X = pd.concat([X, Y_pred], axis=1)
    # n_features += n_classes_nonscored

In [14]:
def train_and_evaluate(X, Y, is_ctl_vehicle=False, is_drug_kfold=True, is_pretrain=False):
    bias_initializer = -Y.mean(axis=0).apply(np.log).values
    bias_initializer = tf.keras.initializers.Constant(bias_initializer)

    Y_pred = np.zeros((train_size, n_classes))
    Y_pred = pd.DataFrame(Y_pred, columns=Y.columns, index=X.index)

    for i in range(n_seeds):
        set_seed(seed=i)

        if is_drug_kfold:
            cv = MultilabelGroupStratifiedKFold(
                n_splits=n_splits, random_state=i, shuffle=shuffle
            )
            cv_split = cv.split(X, Y, groups)
        else:
            cv = MultilabelStratifiedKFold(
                n_splits=n_splits, random_state=i, shuffle=shuffle
            )
            cv_split = cv.split(X, Y)

        for j, (train, valid) in tqdm(enumerate(cv_split)):

            if is_pretrain:
                # model_nonscored_path = f"model_nonscored_seed_{i}_fold_{j}.h5"
                model_path = f"{out_dir}/model_nonscored_seed_{i}_fold_{j}.h5"
                params["pretrained_model_path"] = model_path
            
            # model_path = f"model_seed_{i}_fold_{j}.h5"
            model_path = f"{out_dir}/model_seed_{i}_fold_{j}.h5"

            K.clear_session()
            model = build_model(
                n_features,
                n_classes,
                bias_initializer=bias_initializer,
                # pretrained_model_path=model_nonscored_path,
                **params,
            )

            callbacks = build_callbacks(model_path, factor=factor, patience=patience)

            if cutmix_alpha > 0.0:
                history = model.fit(
                    Cutmix(
                        X.iloc[train],
                        y=Y.iloc[train],
                        batch_size=fit_params["batch_size"],
                        alpha=cutmix_alpha,
                    ),
                    callbacks=callbacks,
                    validation_data=(X.iloc[valid], Y.iloc[valid]),
                    **fit_params,
                )
            elif mixup_alpha > 0.0:
                history = model.fit(
                    Mixup(
                        X.iloc[train],
                        y=Y.iloc[train],
                        batch_size=fit_params["batch_size"],
                        alpha=mixup_alpha,
                    ),
                    callbacks=callbacks,
                    validation_data=(X.iloc[valid], Y.iloc[valid]),
                    **fit_params,
                )
            else:
                history = model.fit(
                    X.iloc[train],
                    Y.iloc[train],
                    callbacks=callbacks,
                    validation_data=(X.iloc[valid], Y.iloc[valid]),
                    **fit_params,
                )

            plot_history(history)

            model.load_weights(model_path)

            Y_pred.iloc[valid] += (
                predict(model, X.iloc[valid], **predict_params) / n_seeds
            )

    # postprocess
    if is_ctl_vehicle:
        Y_pred = Y_pred.append(
            Y_orig[train_features["cp_type"] == "ctl_vehicle"]
        ).sort_index()
        Y = Y_orig.sort_index()
    else:
        Y_pred[train_features["cp_type"] == "ctl_vehicle"] = 0.0

    Y_pred[Y_pred < threshold] = 0.0
    Y_pred[Y_pred > 1.0 - threshold] = 1.0

    oof_loss = score(Y[columns], Y_pred[columns])
    print(f"Our out of folds log loss score is {oof_loss}")

    return oof_loss

In [15]:
DEBUG = True
#DEBUG = False

out_base_dir = "output"
os.makedirs(out_base_dir, exist_ok=True)

# hyperparameters
batch_size = 32
factor = 0.5
n_seeds = 1  # n_seeds = 5
n_splits = 5
patience = 30
shuffle = True
threshold = 1e-05
params = {
    "activation": "elu",
    "kernel_initializer": "he_normal",
    "label_smoothing": 5e-04,
    "lr": 0.03,
    "n_layers": 7,
    "n_units": 256,
    "rate": 0.25,
    "skip": True,
    "stddev": 0.45,
    "other_params": {
        "optimizer": "adam",
        "is_lookahead": False,
        "is_swa": False,
        "is_weightnorm": True,
        "nn_order": "batch-act-drop",
        "drop_act_rate": 0.0,
    },
}
fit_params = {"batch_size": batch_size, "epochs": 1_000, "verbose": 0}
cutmix_alpha = 0.0
mixup_alpha = 0.0
predict_params = {"batch_size": batch_size, "n_iter": 10}

if DEBUG:
    fit_params = {"batch_size": 1024, "epochs": 80, "verbose": 0}
    logger.info(f"### fit_params: {fit_params} ###")
else:
    logger.info(f"### fit_params: {fit_params} ###")


# preprocess
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

[2020-11-18 14:21:53] - ### fit_params: {'batch_size': 1024, 'epochs': 80, 'verbose': 0} ###


# activation, bn, dropout の順序

In [16]:
# train
for nn_order in [
    "batch-act-drop",
    "batch-drop-act",
    "drop-batch-act",
    "drop-act-batch",
    "act-batch-drop",
    "act-drop-batch",
]:

    out_dir = f"{out_base_dir}/{nn_order}"
    os.makedirs(out_dir, exist_ok=True)

    params["other_params"]["nn_order"] = nn_order
    oof_loss = train_and_evaluate(X, Y)
    logger.result(f"nn_order: {params['other_params']['nn_order']}\t{oof_loss}")
    print("-" * 100)

params["other_params"]["nn_order"] = "batch-act-drop"

5it [02:02, 24.57s/it]
nn_order: batch-act-drop	0.015985684071960313
0it [00:00, ?it/s]

Our out of folds log loss score is 0.015985684071960313
----------------------------------------------------------------------------------------------------


5it [01:50, 22.19s/it]
nn_order: batch-drop-act	0.016349560808381864
0it [00:00, ?it/s]

Our out of folds log loss score is 0.016349560808381864
----------------------------------------------------------------------------------------------------


5it [01:40, 20.02s/it]
nn_order: drop-batch-act	0.01644531014749022
0it [00:00, ?it/s]

...

Our out of folds log loss score is 0.016430224215919793
----------------------------------------------------------------------------------------------------


5it [01:54, 22.84s/it]
nn_order: act-batch-drop	0.016053474633091317
0it [00:00, ?it/s]

Our out of folds log loss score is 0.016053474633091317
----------------------------------------------------------------------------------------------------


5it [01:42, 20.52s/it]
nn_order: act-drop-batch	0.016171339817555343


Our out of folds log loss score is 0.016171339817555343
----------------------------------------------------------------------------------------------------


# weight normalization の有無

In [17]:
# train
for is_weightnorm in [True, False]:

    out_dir = f"{out_base_dir}/is_weightnorm_{is_weightnorm}"
    os.makedirs(out_dir, exist_ok=True)

    params["other_params"]["is_weightnorm"] = is_weightnorm
    oof_loss = train_and_evaluate(X, Y)
    logger.result(
        f"is_weightnorm: {params['other_params']['is_weightnorm']}\t{oof_loss}"
    )
    print("-" * 100)

params["other_params"]["is_weightnorm"] = True

5it [02:01, 24.29s/it]
is_weightnorm: True	0.015985588891270556
0it [00:00, ?it/s]

Our out of folds log loss score is 0.015985588891270556
----------------------------------------------------------------------------------------------------


5it [01:52, 22.53s/it]
is_weightnorm: False	0.01698612993024874


Our out of folds log loss score is 0.01698612993024874
----------------------------------------------------------------------------------------------------


# DropActivation の有無

In [18]:
# train
for drop_act_rate in [0.05, 0.25]:

    out_dir = f"{out_base_dir}/DropActivation"
    os.makedirs(out_dir, exist_ok=True)

    params["other_params"]["drop_act_rate"] = drop_act_rate
    oof_loss = train_and_evaluate(X, Y)
    logger.result(
        f"drop_act_rate: {params['other_params']['drop_act_rate']}\t{oof_loss}"
    )
    print("-" * 100)

params["other_params"]["drop_act_rate"] = 0.0

5it [01:45, 21.04s/it]
drop_act_rate: 0.05	0.016411081229058176
0it [00:00, ?it/s]

Our out of folds log loss score is 0.016411081229058176
----------------------------------------------------------------------------------------------------


5it [01:43, 20.68s/it]
drop_act_rate: 0.25	0.017047980939148145


Our out of folds log loss score is 0.017047980939148145
----------------------------------------------------------------------------------------------------


# CutMix, MixUp

In [19]:
# train
for cutmix_alpha in [0.3, 1.0]:

    out_dir = f"{out_base_dir}"
    os.makedirs(out_dir, exist_ok=True)

    oof_loss = train_and_evaluate(X, Y)
    logger.result(f"cutmix_alpha: {cutmix_alpha}\t{oof_loss}")
    print("-" * 100)

cutmix_alpha = 0.0

5it [06:09, 73.94s/it]
cutmix_alpha: 0.3	0.015834075264526248
0it [00:00, ?it/s]

Our out of folds log loss score is 0.015834075264526248
----------------------------------------------------------------------------------------------------


5it [06:34, 78.88s/it]
cutmix_alpha: 1.0	0.01582797283297462


Our out of folds log loss score is 0.01582797283297462
----------------------------------------------------------------------------------------------------


In [20]:
# train
params["stddev"] = 0.0
cutmix_alpha = 1.0

out_dir = f"{out_base_dir}"
os.makedirs(out_dir, exist_ok=True)

oof_loss = train_and_evaluate(X, Y)
logger.result(f"stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

5it [06:24, 76.97s/it]
stddev: 0.0, cutmix_alpha: 1.0	0.01582260091243955


Our out of folds log loss score is 0.01582260091243955
----------------------------------------------------------------------------------------------------


In [21]:
# train
for mixup_alpha in [0.2, 0.5]:

    out_dir = f"{out_base_dir}"
    os.makedirs(out_dir, exist_ok=True)

    oof_loss = train_and_evaluate(X, Y)
    logger.result(f"mixup_alpha: {mixup_alpha}\t{oof_loss}")
    print("-" * 100)

mixup_alpha = 0.0

5it [05:34, 66.82s/it]
mixup_alpha: 0.2	0.01585606317687861
0it [00:00, ?it/s]

Our out of folds log loss score is 0.01585606317687861
----------------------------------------------------------------------------------------------------


5it [05:52, 70.52s/it]
mixup_alpha: 0.5	0.015811547251864327


Our out of folds log loss score is 0.015811547251864327
----------------------------------------------------------------------------------------------------


In [22]:
# train
params["stddev"] = 0.0
mixup_alpha = 0.5

out_dir = f"{out_base_dir}"
os.makedirs(out_dir, exist_ok=True)

oof_loss = train_and_evaluate(X, Y)
logger.result(f"stddev: {params['stddev']}, mixup_alpha: {mixup_alpha}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
mixup_alpha = 0.0

5it [06:24, 76.88s/it]
stddev: 0.0, mixup_alpha: 0.5	0.01595175720154114


Our out of folds log loss score is 0.01595175720154114
----------------------------------------------------------------------------------------------------


# pretrain
- https://www.kaggle.com/c/lish-moa/discussion/195859 より

In [34]:
# train
params["stddev"] = 0.0
cutmix_alpha = 1.0

out_dir = f"{out_base_dir}"

pretrain(X, Y_nonscored)
oof_loss = train_and_evaluate(X, Y, is_pretrain=True)
logger.result(f"pretrain, stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

0it [00:00, ?it/s]

1it [03:32, 212.09s/it]

2it [06:40, 205.05s/it]

3it [09:59, 203.09s/it]

4it [13:12, 200.03s/it]

5it [16:12, 194.56s/it]
stddev: 0.0, cutmix_alpha: 1.0	0.016281214295192914


Our out of folds log loss score is 0.016281214295192914
----------------------------------------------------------------------------------------------------


# cp_type と cp_dose の有無

In [23]:
dtype = {}
index_col = "sig_id"

DATADIR = (
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\input\lish-moa"
)
# DATADIR = r"../input/lish-moa"

_train_features = pd.read_csv(
    f"{DATADIR}/train_features.csv", dtype=dtype, index_col=index_col
)

X = _train_features.copy()
cp_type = {"trt_cp": 0, "ctl_vehicle": 1}
cp_dose = {"D1": 0, "D2": 1}
for _X in [X, X_test]:
    _X["cp_type"] = _X["cp_type"].map(cp_type)
    _X["cp_dose"] = _X["cp_dose"].map(cp_dose)
X = X.select_dtypes("number")

X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

display(X.head())

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"cp_type + cp_dose\t{oof_loss}")
print("-" * 100)

,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680

5it [02:41, 32.37s/it]
cp_type + cp_dose	0.016129993302467588


Our out of folds log loss score is 0.016129993302467588
----------------------------------------------------------------------------------------------------


In [24]:
dtype = {}
index_col = "sig_id"
DATADIR = (
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\input\lish-moa"
)
# DATADIR = r"../input/lish-moa"
_train_features = pd.read_csv(
    f"{DATADIR}/train_features.csv", dtype=dtype, index_col=index_col
)
X = _train_features.copy()
cp_type = {"trt_cp": 0, "ctl_vehicle": 1}
for _X in [X, X_test]:
    _X["cp_type"] = _X["cp_type"].map(cp_type)
X = X.select_dtypes("number")

X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

display(X.head())

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"cp_type\t{oof_loss}")
print("-" * 100)

,cp_type,cp_time,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680,g-681,g

5it [02:40, 32.04s/it]
cp_type	0.016456924425274723


Our out of folds log loss score is 0.016456924425274723
----------------------------------------------------------------------------------------------------


In [25]:
dtype = {}
index_col = "sig_id"
DATADIR = (
    r"C:\Users\81908\jupyter_notebook\poetry_work\tfgpu\01_MoA_compe\input\lish-moa"
)
# DATADIR = r"../input/lish-moa"
_train_features = pd.read_csv(
    f"{DATADIR}/train_features.csv", dtype=dtype, index_col=index_col
)
X = _train_features.copy()
cp_dose = {"D1": 0, "D2": 1}
for _X in [X, X_test]:
    _X["cp_dose"] = _X["cp_dose"].map(cp_dose)
X = X.select_dtypes("number")

X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

display(X.head())

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"cp_dose\t{oof_loss}")
print("-" * 100)

,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,g-6,g-7,g-8,g-9,g-10,g-11,g-12,g-13,g-14,g-15,g-16,g-17,g-18,g-19,g-20,g-21,g-22,g-23,g-24,g-25,g-26,g-27,g-28,g-29,g-30,g-31,g-32,g-33,g-34,g-35,g-36,g-37,g-38,g-39,g-40,g-41,g-42,g-43,g-44,g-45,g-46,g-47,g-48,g-49,g-50,g-51,g-52,g-53,g-54,g-55,g-56,g-57,g-58,g-59,g-60,g-61,g-62,g-63,g-64,g-65,g-66,g-67,g-68,g-69,g-70,g-71,g-72,g-73,g-74,g-75,g-76,g-77,g-78,g-79,g-80,g-81,g-82,g-83,g-84,g-85,g-86,g-87,g-88,g-89,g-90,g-91,g-92,g-93,g-94,g-95,g-96,g-97,g-98,g-99,g-100,g-101,g-102,g-103,g-104,g-105,g-106,g-107,g-108,g-109,g-110,g-111,g-112,g-113,g-114,g-115,g-116,g-117,g-118,g-119,g-120,g-121,g-122,g-123,g-124,g-125,g-126,g-127,g-128,g-129,g-130,g-131,g-132,g-133,g-134,g-135,g-136,g-137,g-138,g-139,g-140,g-141,g-142,g-143,g-144,g-145,g-146,g-147,g-148,g-149,g-150,g-151,g-152,g-153,g-154,g-155,g-156,g-157,g-158,g-159,g-160,g-161,g-162,g-163,g-164,g-165,g-166,g-167,g-168,g-169,g-170,g-171,g-172,g-173,g-174,g-175,g-176,g-177,g-178,g-179,g-180,g-181,g-182,g-183,g-184,g-185,g-186,g-187,g-188,g-189,g-190,g-191,g-192,g-193,g-194,g-195,g-196,g-197,g-198,g-199,g-200,g-201,g-202,g-203,g-204,g-205,g-206,g-207,g-208,g-209,g-210,g-211,g-212,g-213,g-214,g-215,g-216,g-217,g-218,g-219,g-220,g-221,g-222,g-223,g-224,g-225,g-226,g-227,g-228,g-229,g-230,g-231,g-232,g-233,g-234,g-235,g-236,g-237,g-238,g-239,g-240,g-241,g-242,g-243,g-244,g-245,g-246,g-247,g-248,g-249,g-250,g-251,g-252,g-253,g-254,g-255,g-256,g-257,g-258,g-259,g-260,g-261,g-262,g-263,g-264,g-265,g-266,g-267,g-268,g-269,g-270,g-271,g-272,g-273,g-274,g-275,g-276,g-277,g-278,g-279,g-280,g-281,g-282,g-283,g-284,g-285,g-286,g-287,g-288,g-289,g-290,g-291,g-292,g-293,g-294,g-295,g-296,g-297,g-298,g-299,g-300,g-301,g-302,g-303,g-304,g-305,g-306,g-307,g-308,g-309,g-310,g-311,g-312,g-313,g-314,g-315,g-316,g-317,g-318,g-319,g-320,g-321,g-322,g-323,g-324,g-325,g-326,g-327,g-328,g-329,g-330,g-331,g-332,g-333,g-334,g-335,g-336,g-337,g-338,g-339,g-340,g-341,g-342,g-343,g-344,g-345,g-346,g-347,g-348,g-349,g-350,g-351,g-352,g-353,g-354,g-355,g-356,g-357,g-358,g-359,g-360,g-361,g-362,g-363,g-364,g-365,g-366,g-367,g-368,g-369,g-370,g-371,g-372,g-373,g-374,g-375,g-376,g-377,g-378,g-379,g-380,g-381,g-382,g-383,g-384,g-385,g-386,g-387,g-388,g-389,g-390,g-391,g-392,g-393,g-394,g-395,g-396,g-397,g-398,g-399,g-400,g-401,g-402,g-403,g-404,g-405,g-406,g-407,g-408,g-409,g-410,g-411,g-412,g-413,g-414,g-415,g-416,g-417,g-418,g-419,g-420,g-421,g-422,g-423,g-424,g-425,g-426,g-427,g-428,g-429,g-430,g-431,g-432,g-433,g-434,g-435,g-436,g-437,g-438,g-439,g-440,g-441,g-442,g-443,g-444,g-445,g-446,g-447,g-448,g-449,g-450,g-451,g-452,g-453,g-454,g-455,g-456,g-457,g-458,g-459,g-460,g-461,g-462,g-463,g-464,g-465,g-466,g-467,g-468,g-469,g-470,g-471,g-472,g-473,g-474,g-475,g-476,g-477,g-478,g-479,g-480,g-481,g-482,g-483,g-484,g-485,g-486,g-487,g-488,g-489,g-490,g-491,g-492,g-493,g-494,g-495,g-496,g-497,g-498,g-499,g-500,g-501,g-502,g-503,g-504,g-505,g-506,g-507,g-508,g-509,g-510,g-511,g-512,g-513,g-514,g-515,g-516,g-517,g-518,g-519,g-520,g-521,g-522,g-523,g-524,g-525,g-526,g-527,g-528,g-529,g-530,g-531,g-532,g-533,g-534,g-535,g-536,g-537,g-538,g-539,g-540,g-541,g-542,g-543,g-544,g-545,g-546,g-547,g-548,g-549,g-550,g-551,g-552,g-553,g-554,g-555,g-556,g-557,g-558,g-559,g-560,g-561,g-562,g-563,g-564,g-565,g-566,g-567,g-568,g-569,g-570,g-571,g-572,g-573,g-574,g-575,g-576,g-577,g-578,g-579,g-580,g-581,g-582,g-583,g-584,g-585,g-586,g-587,g-588,g-589,g-590,g-591,g-592,g-593,g-594,g-595,g-596,g-597,g-598,g-599,g-600,g-601,g-602,g-603,g-604,g-605,g-606,g-607,g-608,g-609,g-610,g-611,g-612,g-613,g-614,g-615,g-616,g-617,g-618,g-619,g-620,g-621,g-622,g-623,g-624,g-625,g-626,g-627,g-628,g-629,g-630,g-631,g-632,g-633,g-634,g-635,g-636,g-637,g-638,g-639,g-640,g-641,g-642,g-643,g-644,g-645,g-646,g-647,g-648,g-649,g-650,g-651,g-652,g-653,g-654,g-655,g-656,g-657,g-658,g-659,g-660,g-661,g-662,g-663,g-664,g-665,g-666,g-667,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-675,g-676,g-677,g-678,g-679,g-680,g-681,g

5it [02:49, 33.83s/it]
cp_dose	0.016672892182358734


Our out of folds log loss score is 0.016672892182358734
----------------------------------------------------------------------------------------------------


# 特徴量指定

In [16]:
start_predictors = [
    "g-0",
    "g-7",
    "g-8",
    "g-10",
    "g-13",
    "g-17",
    "g-20",
    "g-22",
    "g-24",
    "g-26",
    "g-28",
    "g-29",
    "g-30",
    "g-31",
    "g-32",
    "g-34",
    "g-35",
    "g-36",
    "g-37",
    "g-38",
    "g-39",
    "g-41",
    "g-46",
    "g-48",
    "g-50",
    "g-51",
    "g-52",
    "g-55",
    "g-58",
    "g-59",
    "g-61",
    "g-62",
    "g-63",
    "g-65",
    "g-66",
    "g-67",
    "g-68",
    "g-70",
    "g-72",
    "g-74",
    "g-75",
    "g-79",
    "g-83",
    "g-84",
    "g-85",
    "g-86",
    "g-90",
    "g-91",
    "g-94",
    "g-95",
    "g-96",
    "g-97",
    "g-98",
    "g-100",
    "g-102",
    "g-105",
    "g-106",
    "g-112",
    "g-113",
    "g-114",
    "g-116",
    "g-121",
    "g-123",
    "g-126",
    "g-128",
    "g-131",
    "g-132",
    "g-134",
    "g-135",
    "g-138",
    "g-139",
    "g-140",
    "g-142",
    "g-144",
    "g-145",
    "g-146",
    "g-147",
    "g-148",
    "g-152",
    "g-155",
    "g-157",
    "g-158",
    "g-160",
    "g-163",
    "g-164",
    "g-165",
    "g-170",
    "g-173",
    "g-174",
    "g-175",
    "g-177",
    "g-178",
    "g-181",
    "g-183",
    "g-185",
    "g-186",
    "g-189",
    "g-192",
    "g-194",
    "g-195",
    "g-196",
    "g-197",
    "g-199",
    "g-201",
    "g-202",
    "g-206",
    "g-208",
    "g-210",
    "g-213",
    "g-214",
    "g-215",
    "g-220",
    "g-226",
    "g-228",
    "g-229",
    "g-235",
    "g-238",
    "g-241",
    "g-242",
    "g-243",
    "g-244",
    "g-245",
    "g-248",
    "g-250",
    "g-251",
    "g-254",
    "g-257",
    "g-259",
    "g-261",
    "g-266",
    "g-270",
    "g-271",
    "g-272",
    "g-275",
    "g-278",
    "g-282",
    "g-287",
    "g-288",
    "g-289",
    "g-291",
    "g-293",
    "g-294",
    "g-297",
    "g-298",
    "g-301",
    "g-303",
    "g-304",
    "g-306",
    "g-308",
    "g-309",
    "g-310",
    "g-311",
    "g-314",
    "g-315",
    "g-316",
    "g-317",
    "g-320",
    "g-321",
    "g-322",
    "g-327",
    "g-328",
    "g-329",
    "g-332",
    "g-334",
    "g-335",
    "g-336",
    "g-337",
    "g-339",
    "g-342",
    "g-344",
    "g-349",
    "g-350",
    "g-351",
    "g-353",
    "g-354",
    "g-355",
    "g-357",
    "g-359",
    "g-360",
    "g-364",
    "g-365",
    "g-366",
    "g-367",
    "g-368",
    "g-369",
    "g-374",
    "g-375",
    "g-377",
    "g-379",
    "g-385",
    "g-386",
    "g-390",
    "g-392",
    "g-393",
    "g-400",
    "g-402",
    "g-406",
    "g-407",
    "g-409",
    "g-410",
    "g-411",
    "g-414",
    "g-417",
    "g-418",
    "g-421",
    "g-423",
    "g-424",
    "g-427",
    "g-429",
    "g-431",
    "g-432",
    "g-433",
    "g-434",
    "g-437",
    "g-439",
    "g-440",
    "g-443",
    "g-449",
    "g-458",
    "g-459",
    "g-460",
    "g-461",
    "g-464",
    "g-467",
    "g-468",
    "g-470",
    "g-473",
    "g-477",
    "g-478",
    "g-479",
    "g-484",
    "g-485",
    "g-486",
    "g-488",
    "g-489",
    "g-491",
    "g-494",
    "g-496",
    "g-498",
    "g-500",
    "g-503",
    "g-504",
    "g-506",
    "g-508",
    "g-509",
    "g-512",
    "g-522",
    "g-529",
    "g-531",
    "g-534",
    "g-539",
    "g-541",
    "g-546",
    "g-551",
    "g-553",
    "g-554",
    "g-559",
    "g-561",
    "g-562",
    "g-565",
    "g-568",
    "g-569",
    "g-574",
    "g-577",
    "g-578",
    "g-586",
    "g-588",
    "g-590",
    "g-594",
    "g-595",
    "g-596",
    "g-597",
    "g-599",
    "g-600",
    "g-603",
    "g-607",
    "g-615",
    "g-618",
    "g-619",
    "g-620",
    "g-625",
    "g-628",
    "g-629",
    "g-632",
    "g-634",
    "g-635",
    "g-636",
    "g-638",
    "g-639",
    "g-641",
    "g-643",
    "g-644",
    "g-645",
    "g-646",
    "g-647",
    "g-648",
    "g-663",
    "g-664",
    "g-665",
    "g-668",
    "g-669",
    "g-670",
    "g-671",
    "g-672",
    "g-673",
    "g-674",
    "g-677",
    "g-678",
    "g-680",
    "g-683",
    "g-689",
    "g-691",
    "g-693",
    "g-695",
    "g-701",
    "g-702",
    "g-703",
    "g-704",
    "g-705",
    "g-706",
    "g-708",
    "g-711",
    "g-712",
    "g-720",
    "g-721",
    "g-723",
    "g-724",
    "g-726",
    "g-728",
    "g-731",
    "g-733",
    "g-738",
    "g-739",
    "g-742",
    "g-743",
    "g-744",
    "g-745",
    "g-749",
    "g-750",
    "g-752",
    "g-760",
    "g-761",
    "g-764",
    "g-766",
    "g-768",
    "g-770",
    "g-771",
    "c-0",
    "c-1",
    "c-2",
    "c-3",
    "c-4",
    "c-5",
    "c-6",
    "c-7",
    "c-8",
    "c-9",
    "c-10",
    "c-11",
    "c-12",
    "c-13",
    "c-14",
    "c-15",
    "c-16",
    "c-17",
    "c-18",
    "c-19",
    "c-20",
    "c-21",
    "c-22",
    "c-23",
    "c-24",
    "c-25",
    "c-26",
    "c-27",
    "c-28",
    "c-29",
    "c-30",
    "c-31",
    "c-32",
    "c-33",
    "c-34",
    "c-35",
    "c-36",
    "c-37",
    "c-38",
    "c-39",
    "c-40",
    "c-41",
    "c-42",
    "c-43",
    "c-44",
    "c-45",
    "c-46",
    "c-47",
    "c-48",
    "c-49",
    "c-50",
    "c-51",
    "c-52",
    "c-53",
    "c-54",
    "c-55",
    "c-56",
    "c-57",
    "c-58",
    "c-59",
    "c-60",
    "c-61",
    "c-62",
    "c-63",
    "c-64",
    "c-65",
    "c-66",
    "c-67",
    "c-68",
    "c-69",
    "c-70",
    "c-71",
    "c-72",
    "c-73",
    "c-74",
    "c-75",
    "c-76",
    "c-77",
    "c-78",
    "c-79",
    "c-80",
    "c-81",
    "c-82",
    "c-83",
    "c-84",
    "c-85",
    "c-86",
    "c-87",
    "c-88",
    "c-89",
    "c-90",
    "c-91",
    "c-92",
    "c-93",
    "c-94",
    "c-95",
    "c-96",
    "c-97",
    "c-98",
    "c-99",
]
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X = X.loc[:,start_predictors]

X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

display(X.head())

# train
params["stddev"] = 0.0
cutmix_alpha = 1.0

out_dir = f"{out_base_dir}"

oof_loss = train_and_evaluate(X, Y)
logger.result(f"start_predictors, stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

,g-0,g-7,g-8,g-10,g-13,g-17,g-20,g-22,g-24,g-26,g-28,g-29,g-30,g-31,g-32,g-34,g-35,g-36,g-37,g-38,g-39,g-41,g-46,g-48,g-50,g-51,g-52,g-55,g-58,g-59,g-61,g-62,g-63,g-65,g-66,g-67,g-68,g-70,g-72,g-74,g-75,g-79,g-83,g-84,g-85,g-86,g-90,g-91,g-94,g-95,g-96,g-97,g-98,g-100,g-102,g-105,g-106,g-112,g-113,g-114,g-116,g-121,g-123,g-126,g-128,g-131,g-132,g-134,g-135,g-138,g-139,g-140,g-142,g-144,g-145,g-146,g-147,g-148,g-152,g-155,g-157,g-158,g-160,g-163,g-164,g-165,g-170,g-173,g-174,g-175,g-177,g-178,g-181,g-183,g-185,g-186,g-189,g-192,g-194,g-195,g-196,g-197,g-199,g-201,g-202,g-206,g-208,g-210,g-213,g-214,g-215,g-220,g-226,g-228,g-229,g-235,g-238,g-241,g-242,g-243,g-244,g-245,g-248,g-250,g-251,g-254,g-257,g-259,g-261,g-266,g-270,g-271,g-272,g-275,g-278,g-282,g-287,g-288,g-289,g-291,g-293,g-294,g-297,g-298,g-301,g-303,g-304,g-306,g-308,g-309,g-310,g-311,g-314,g-315,g-316,g-317,g-320,g-321,g-322,g-327,g-328,g-329,g-332,g-334,g-335,g-336,g-337,g-339,g-342,g-344,g-349,g-350,g-351,g-353,g-354,g-355,g-357,g-359,g-360,g-364,g-365,g-366,g-367,g-368,g-369,g-374,g-375,g-377,g-379,g-385,g-386,g-390,g-392,g-393,g-400,g-402,g-406,g-407,g-409,g-410,g-411,g-414,g-417,g-418,g-421,g-423,g-424,g-427,g-429,g-431,g-432,g-433,g-434,g-437,g-439,g-440,g-443,g-449,g-458,g-459,g-460,g-461,g-464,g-467,g-468,g-470,g-473,g-477,g-478,g-479,g-484,g-485,g-486,g-488,g-489,g-491,g-494,g-496,g-498,g-500,g-503,g-504,g-506,g-508,g-509,g-512,g-522,g-529,g-531,g-534,g-539,g-541,g-546,g-551,g-553,g-554,g-559,g-561,g-562,g-565,g-568,g-569,g-574,g-577,g-578,g-586,g-588,g-590,g-594,g-595,g-596,g-597,g-599,g-600,g-603,g-607,g-615,g-618,g-619,g-620,g-625,g-628,g-629,g-632,g-634,g-635,g-636,g-638,g-639,g-641,g-643,g-644,g-645,g-646,g-647,g-648,g-663,g-664,g-665,g-668,g-669,g-670,g-671,g-672,g-673,g-674,g-677,g-678,g-680,g-683,g-689,g-691,g-693,g-695,g-701,g-702,g-703,g-704,g-705,g-706,g-708,g-711,g-712,g-720,g-721,g-723,g-724,g-726,g-728,g-731,g-733,g-738,g-739,g-742,g-743,g-744,g-745,g-749,g-750,g-752,g-760,g-761,g-764,g-766,g-768,g-770,g-771,c-0,c-1,c-2,c-3,c-4,c-5,c-6,c-7,c-8,c-9,c-10,c-11,c-12,c-13,c-14,c-15,c-16,c-17,c-18,c-19,c-20,c-21,c-22,c-23,c-24,c-25,c-26,c-27,c-28,c-29,c-30,c-31,c-32,c-33,c-34,c-35,c-36,c-37,c-38,c-39,c-40,c-41,c-42,c-43,c-44,c-45,c-46,c-47,c-48,c-49,c-50,c-51,c-52,c-53,c-54,c-55,c-56,c-57,c-58,c-59,c-60,c-61,c-62,c-63,c-64,c-65,c-66,c-67,c-68,c-69,c-70,c-71,c-72,c-73,c-74,c-75,c-76,c-77,c-78,c-79,c-80,c-81,c-82,c-83,c-84,c-85,c-86,c-87,c-88,c-89,c-90,c-91,c-92,c-93,c-94,c-95,c-96,c-97,c-98,c-99,c-mean,c-std,c-kurtosis,c-skew,g-mean,g-std,g-kurtosis,g-skew
sig_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
id_000644bb2,1.0620,-0.0326,0.5548,1.1830,-0.4015,0.6342,-0.5688,-1.1880,0.4393,0.1907,-0.2853,0.5819,0.2934,-0.5584,-0.0916,-0.1537,0.2198,0.2965,-0.5055,-0.5119,-0.2162,-0.2566,-0.1143,0.0350,-0.1221,-0.4927,0.6119,-0.1605,0.5790,0.9588,-0.13000,0.6611,-0.3136,0.0251,0.1670,-0.4195,0.6551,-0.7047,0.0314,-0.3406,-0.6332,0.7348,-2.00935,1.7320,0.0190,-0.11880,0.2048,-1.1230,0.7017,-1.0390,0.0958,0.45270,0.5455,-0.5364,-0.7768,-1.2560,0.1593,1.2670,0.4734,-0.9902,-0.1188,1.2880,0.0486,-0.1810,-0.5130,-0.7349,0.3116,0.3539,0.28330,-0.4646,-0.4940,0.5297,-1.1870,1.13600,-0.1758,0.3858,-1.2530,-0.1779,-0.1973,0.42790,-0.1091,-0.7799,-0.3239,-0.5592,-0.4386,0.0590,-0.4194,1.6820,-0.2471,-1.4130,-0.2256,-0.6217,-0.3284,-0.2786,-0.5274,0.5698,-0.1078,0.0917,-0.1509,0.7374,-0.7406,-0.5210,0.8366,0.4171,0.2383,0.5875,-0.8059,-0.3744,0.1093,0.7356,-1.2170,-1.03200,-0.6200,1.1080,-0.0549,0.1218,0.2143,0.5628,0.4498,-1.2160,0.1905,0.3205,-0.3072,0.3802,0.87

5it [29:15, 351.13s/it]
start_predictors, stddev: 0.0, cutmix_alpha: 1.0	0.015933981452463882


Our out of folds log loss score is 0.015933981452463882
----------------------------------------------------------------------------------------------------


# 前処理・特徴エンジニアリング

In [16]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 600,
    "pca_n_components_c": 50,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:22, 28.52s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 600, 'pca_n_components_c': 50, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015968692570968685


Our out of folds log loss score is 0.015968692570968685
----------------------------------------------------------------------------------------------------


In [17]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 600,
    "pca_n_components_c": 50,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:17, 27.51s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 600, 'pca_n_components_c': 50, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.01597841556717841


Our out of folds log loss score is 0.01597841556717841
----------------------------------------------------------------------------------------------------


In [21]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 70,
    "pca_n_components_c": 10,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:59, 23.90s/it]
preprocess_params: {'is_clip': True, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 70, 'pca_n_components_c': 10, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01591545449758546


Our out of folds log loss score is 0.01591545449758546
----------------------------------------------------------------------------------------------------


In [22]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 70,
    "pca_n_components_c": 10,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:58, 23.79s/it]
preprocess_params: {'is_clip': True, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 70, 'pca_n_components_c': 10, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.01593495103980279


Our out of folds log loss score is 0.01593495103980279
----------------------------------------------------------------------------------------------------


In [18]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:50, 22.13s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.016057313121728308


Our out of folds log loss score is 0.016057313121728308
----------------------------------------------------------------------------------------------------


In [19]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:50, 22.12s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.016057310025097284


Our out of folds log loss score is 0.016057310025097284
----------------------------------------------------------------------------------------------------


In [16]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

# train pretrainあり
params["stddev"] = 0.0
cutmix_alpha = 1.0

out_dir = f"{out_base_dir}"

pretrain(X, Y_nonscored)
oof_loss = train_and_evaluate(X, Y)
logger.result(f"pretrain, stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}, preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

5it [00:55, 11.11s/it]
0it [00:00, ?it/s]

1it [01:09, 69.92s/it]

2it [02:15, 68.75s/it]

3it [03:18, 66.77s/it]

4it [04:17, 64.70s/it]

5it [05:16, 63.34s/it]
pretrain, stddev: 0.0, cutmix_alpha: 1.0, preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015954196787044485


Our out of folds log loss score is 0.015954196787044485
----------------------------------------------------------------------------------------------------


In [26]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:19, 27.83s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015859584553864197


Our out of folds log loss score is 0.015859584553864197
----------------------------------------------------------------------------------------------------


In [27]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:23, 28.63s/it]
preprocess_params: {'is_clip': True, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015879353806231383


Our out of folds log loss score is 0.015879353806231383
----------------------------------------------------------------------------------------------------


In [28]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:19, 27.88s/it]
preprocess_params: {'is_clip': False, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015910452327254205


Our out of folds log loss score is 0.015910452327254205
----------------------------------------------------------------------------------------------------


In [29]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:18, 27.73s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015923571165005597


Our out of folds log loss score is 0.015923571165005597
----------------------------------------------------------------------------------------------------


In [30]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": True,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:39, 31.86s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': True, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015875933071122197


Our out of folds log loss score is 0.015875933071122197
----------------------------------------------------------------------------------------------------


In [31]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": True,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:22, 28.52s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': True, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01590171192115532


Our out of folds log loss score is 0.01590171192115532
----------------------------------------------------------------------------------------------------


In [32]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": True,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:42, 32.56s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': True, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01589523748615384


Our out of folds log loss score is 0.01589523748615384
----------------------------------------------------------------------------------------------------


In [33]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 70,
    "pca_n_components_c": 10,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:05, 25.17s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 70, 'pca_n_components_c': 10, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01592401508486589


Our out of folds log loss score is 0.01592401508486589
----------------------------------------------------------------------------------------------------


In [34]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 600,
    "pca_n_components_c": 50,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:23, 28.76s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 600, 'pca_n_components_c': 50, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015968650425839746


Our out of folds log loss score is 0.015968650425839746
----------------------------------------------------------------------------------------------------


In [35]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": True,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:06, 25.30s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': True, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015897730105485824


Our out of folds log loss score is 0.015897730105485824
----------------------------------------------------------------------------------------------------


In [36]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:51, 22.35s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01605731185773959


Our out of folds log loss score is 0.01605731185773959
----------------------------------------------------------------------------------------------------


In [37]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:52, 22.41s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.016057313096525305


Our out of folds log loss score is 0.016057313096525305
----------------------------------------------------------------------------------------------------


In [38]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 1000,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:52, 22.43s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 1000, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.016059854792655034


Our out of folds log loss score is 0.016059854792655034
----------------------------------------------------------------------------------------------------


In [20]:
# preprocess
preprocess_params = {
    "is_clip": False,
    "is_stat": False,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 1000,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [01:51, 22.26s/it]
preprocess_params: {'is_clip': False, 'is_stat': False, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 1000, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.0160598550625519


Our out of folds log loss score is 0.0160598550625519
----------------------------------------------------------------------------------------------------


In [39]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": True,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 600,
    "pca_n_components_c": 50,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:17, 27.55s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': True, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 600, 'pca_n_components_c': 50, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.016223505217168577


Our out of folds log loss score is 0.016223505217168577
----------------------------------------------------------------------------------------------------


In [40]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": True,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 600,
    "pca_n_components_c": 50,
    "is_scaling": False,
    "quantile_n_components": 100,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.8,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:00, 24.15s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': True, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 600, 'pca_n_components_c': 50, 'is_scaling': False, 'quantile_n_components': 100, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.8}	0.016216804735043187


Our out of folds log loss score is 0.016216804735043187
----------------------------------------------------------------------------------------------------


In [41]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": True,
    "is_c_squared": True,
    "is_c_abs": True,
    "is_g_valid": True,
    "pca_n_components_g": 70,
    "pca_n_components_c": 10,
    "is_scaling": True,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [03:02, 36.47s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': True, 'is_c_squared': True, 'is_c_abs': True, 'is_g_valid': True, 'pca_n_components_g': 70, 'pca_n_components_c': 10, 'is_scaling': True, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.016031807965636104


Our out of folds log loss score is 0.016031807965636104
----------------------------------------------------------------------------------------------------


In [42]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": True,
    "is_c_squared": True,
    "is_c_abs": True,
    "is_g_valid": True,
    "pca_n_components_g": 70,
    "pca_n_components_c": 10,
    "is_scaling": True,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [03:11, 38.27s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': True, 'is_c_squared': True, 'is_c_abs': True, 'is_g_valid': True, 'pca_n_components_g': 70, 'pca_n_components_c': 10, 'is_scaling': True, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.015961098308942972


Our out of folds log loss score is 0.015961098308942972
----------------------------------------------------------------------------------------------------


In [43]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": True,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:10, 26.19s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': True, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015823361466178194


Our out of folds log loss score is 0.015823361466178194
----------------------------------------------------------------------------------------------------


In [44]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": True,
    "is_c_abs": True,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:25, 29.03s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': True, 'is_c_abs': True, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015859569927163867


Our out of folds log loss score is 0.015859569927163867
----------------------------------------------------------------------------------------------------


In [45]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": True,
    "is_c_abs": True,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

5it [02:24, 28.88s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': True, 'is_c_abs': True, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015867548727980865


Our out of folds log loss score is 0.015867548727980865
----------------------------------------------------------------------------------------------------


# ctl_vehicle 行の削除
- X,Yの行数変わるから最後に実行すること

In [46]:
out_dir = out_base_dir

is_ctl_vehicle = True

X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
Y_orig = Y.copy()
if is_ctl_vehicle:
    X = X[train_features["cp_type"] != "ctl_vehicle"]
    Y = Y[train_features["cp_type"] != "ctl_vehicle"]
    groups = groups[train_features["cp_type"] != "ctl_vehicle"]
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

oof_loss = train_and_evaluate(X, Y, is_ctl_vehicle=is_ctl_vehicle)
logger.result(f"del ctl_vehicle\t{oof_loss}")
print("-" * 100)

5it [02:11, 26.29s/it]
del ctl_vehicle	0.01593627873622767


Our out of folds log loss score is 0.01593627873622767
----------------------------------------------------------------------------------------------------


In [47]:
out_dir = out_base_dir

is_ctl_vehicle = True

X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
Y_orig = Y.copy()
if is_ctl_vehicle:
    X = X[train_features["cp_type"] != "ctl_vehicle"]
    Y = Y[train_features["cp_type"] != "ctl_vehicle"]
    groups = groups[train_features["cp_type"] != "ctl_vehicle"]
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

params["stddev"] = 0.0
cutmix_alpha = 1.0

oof_loss = train_and_evaluate(X, Y, is_ctl_vehicle=is_ctl_vehicle)
logger.result(f"del ctl_vehicles, tddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}\t{oof_loss}")
print("-" * 100)

5it [06:38, 79.74s/it]
del ctl_vehicles, tddev: 0.0, cutmix_alpha: 1.0	0.015870570934332473


Our out of folds log loss score is 0.015870570934332473
----------------------------------------------------------------------------------------------------


# 組み合わせ

In [48]:
out_dir = out_base_dir

is_ctl_vehicle = True

X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
Y_orig = Y.copy()
if is_ctl_vehicle:
    X = X[train_features["cp_type"] != "ctl_vehicle"]
    Y = Y[train_features["cp_type"] != "ctl_vehicle"]
    groups = groups[train_features["cp_type"] != "ctl_vehicle"]

params["stddev"] = 0.0
cutmix_alpha = 1.0
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

oof_loss = train_and_evaluate(X, Y, is_ctl_vehicle=is_ctl_vehicle)
logger.result(
    f"del ctl_vehicle, stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}, preprocess_params: {preprocess_params}\t{oof_loss}"
)
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

5it [06:32, 78.42s/it]
del ctl_vehicle, stddev: 0.0, cutmix_alpha: 1.0, preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.01583858676430594


Our out of folds log loss score is 0.01583858676430594
----------------------------------------------------------------------------------------------------


In [49]:
out_dir = out_base_dir

is_ctl_vehicle = True

X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
Y_orig = Y.copy()
if is_ctl_vehicle:
    X = X[train_features["cp_type"] != "ctl_vehicle"]
    Y = Y[train_features["cp_type"] != "ctl_vehicle"]
    groups = groups[train_features["cp_type"] != "ctl_vehicle"]

params["stddev"] = 0.0
cutmix_alpha = 1.0
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": False,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": False,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

oof_loss = train_and_evaluate(X, Y, is_ctl_vehicle=is_ctl_vehicle)
logger.result(
    f"del ctl_vehicle, stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}, preprocess_params: {preprocess_params}\t{oof_loss}"
)
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

5it [06:29, 77.99s/it]
del ctl_vehicle, stddev: 0.0, cutmix_alpha: 1.0, preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': False, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': False, 'variance_threshold': 0.0}	0.01583858652263073


Our out of folds log loss score is 0.01583858652263073
----------------------------------------------------------------------------------------------------


# no DEBUG

In [16]:
#DEBUG = True
DEBUG = False

out_base_dir = "output"
os.makedirs(out_base_dir, exist_ok=True)

# hyperparameters
batch_size = 32
factor = 0.5
n_seeds = 1  # n_seeds = 5
n_splits = 5
patience = 30
shuffle = True
threshold = 1e-05
params = {
    "activation": "elu",
    "kernel_initializer": "he_normal",
    "label_smoothing": 5e-04,
    "lr": 0.03,
    "n_layers": 7,
    "n_units": 256,
    "rate": 0.25,
    "skip": True,
    "stddev": 0.45,
    "other_params": {
        "optimizer": "adabelief",
        "is_lookahead": False,
        "is_swa": False,
        "is_weightnorm": True,
        "nn_order": "batch-act-drop",
        "drop_act_rate": 0.0,
    },
}
fit_params = {"batch_size": batch_size, "epochs": 1_000, "verbose": 0}
cutmix_alpha = 0.0
mixup_alpha = 0.0
predict_params = {"batch_size": batch_size, "n_iter": 10}

if DEBUG:
    fit_params = {"batch_size": 1024, "epochs": 80, "verbose": 0}
    logger.info(f"### fit_params: {fit_params} ###")
else:
    logger.info(f"### fit_params: {fit_params} ###")


# preprocess
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(X, Y, Y_nonscored, X_test)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

[2020-11-11 21:40:35] - ### fit_params: {'batch_size': 32, 'epochs': 1000, 'verbose': 0} ###


In [17]:
# train
mixup_alpha = 0.5

out_dir = f"{out_base_dir}"
os.makedirs(out_dir, exist_ok=True)

oof_loss = train_and_evaluate(X, Y)
logger.result(f"mixup_alpha: {mixup_alpha}, params: {params}, fit_params: {fit_params}\t{oof_loss}")
print("-" * 100)

mixup_alpha = 0.0

0it [00:00, ?it/s]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



1it [08:02, 482.57s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



2it [17:19, 504.97s/it]

...

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



4it [33:08, 485.44s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



5it [40:45, 489.16s/it]
mixup_alpha: 0.5, params: {'activation': 'elu', 'kernel_initializer': 'he_normal', 'label_smoothing': 0.0005, 'lr': 0.03, 'n_layers': 7, 'n_units': 256, 'rate': 0.25, 'skip': True, 'stddev': 0.45, 'other_params': {'optimizer': 'adabelief', 'is_lookahead': False, 'is_swa': False, 'is_weightnorm': True, 'nn_order': 'batch-act-drop', 'drop_act_rate': 0.0}}, fit_params: {'batch_size': 32, 'epochs': 1000, 'verbose': 0}	0.015857352552190205


Our out of folds log loss score is 0.015857352552190205
----------------------------------------------------------------------------------------------------


In [18]:
# train
params["stddev"] = 0.0
cutmix_alpha = 1.0

out_dir = f"{out_base_dir}"
os.makedirs(out_dir, exist_ok=True)

oof_loss = train_and_evaluate(X, Y)
logger.result(f"stddev: {params['stddev']}, cutmix_alpha: {cutmix_alpha}, params: {params}, fit_params: {fit_params}\t{oof_loss}")
print("-" * 100)

params["stddev"] = 0.45
cutmix_alpha = 0.0

0it [00:00, ?it/s]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



1it [10:03, 603.67s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



2it [21:23, 626.58s/it]

...

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



4it [39:48, 590.45s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



5it [47:07, 565.48s/it]
stddev: 0.0, cutmix_alpha: 1.0, params: {'activation': 'elu', 'kernel_initializer': 'he_normal', 'label_smoothing': 0.0005, 'lr': 0.03, 'n_layers': 7, 'n_units': 256, 'rate': 0.25, 'skip': True, 'stddev': 0.0, 'other_params': {'optimizer': 'adabelief', 'is_lookahead': False, 'is_swa': False, 'is_weightnorm': True, 'nn_order': 'batch-act-drop', 'drop_act_rate': 0.0}}, fit_params: {'batch_size': 32, 'epochs': 1000, 'verbose': 0}	0.01578128754073746


Our out of folds log loss score is 0.01578128754073746
----------------------------------------------------------------------------------------------------


In [19]:
# preprocess
preprocess_params = {
    "is_clip": True,
    "is_stat": True,
    "is_add_nonscored": False,
    "is_c_squared": False,
    "is_c_abs": True,
    "is_g_valid": False,
    "pca_n_components_g": 0,
    "pca_n_components_c": 0,
    "is_scaling": False,
    "quantile_n_components": 0,
    "fe_cluster_n_clusters_g": 0,
    "fe_cluster_n_clusters_c": 0,
    "is_fit_train_only": True,
    "variance_threshold": 0.0,
}
X, Y, Y_nonscored, train_features, columns, groups, X_test = load_data()
X, Y, Y_nonscored = preprocess(
    X, Y, Y_nonscored, X_test, preprocess_params=preprocess_params
)
train_size, n_features = X.shape
_, n_classes_nonscored = Y_nonscored.shape
_, n_classes = Y.shape

out_dir = out_base_dir

oof_loss = train_and_evaluate(X, Y)
logger.result(f"preprocess_params: {preprocess_params}\t{oof_loss}")
print("-" * 100)

0it [00:00, ?it/s]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



1it [06:33, 393.35s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



2it [12:04, 374.60s/it]

...

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



4it [25:55, 398.29s/it]

Please check your arguments if you have upgraded adabelief-tf from version 0.0.1.
Modifications to default arguments:
                           eps  weight_decouple    rectify
-----------------------  -----  -----------------  -------------
adabelief-tf=0.0.1       1e-08  Not supported      Not supported
Current version (0.1.0)  1e-14  supported          default: True
For a complete table of recommended hyperparameters, see
https://github.com/juntang-zhuang/Adabelief-Optimizer



5it [35:23, 424.71s/it]
preprocess_params: {'is_clip': True, 'is_stat': True, 'is_add_nonscored': False, 'is_c_squared': False, 'is_c_abs': True, 'is_g_valid': False, 'pca_n_components_g': 0, 'pca_n_components_c': 0, 'is_scaling': False, 'quantile_n_components': 0, 'fe_cluster_n_clusters_g': 0, 'fe_cluster_n_clusters_c': 0, 'is_fit_train_only': True, 'variance_threshold': 0.0}	0.015870346833396873


Our out of folds log loss score is 0.015870346833396873
----------------------------------------------------------------------------------------------------
